![](https://media.licdn.com/dms/image/v2/D563DAQEDDNaX16uEjg/image-scale_191_1128/B56ZVsgHQmHQAc-/0/1741282113571/hull_tactical_asset_allocation_htaa__cover?e=2147483647&v=beta&t=paSlb8wX5F2ELEHk3IaRAPLlY9ZTPO3UJhNZbFa60kY)

📊 Description: This project builds a complete end-to-end machine learning pipeline to predict forward returns in financial markets.

🔍 Objective: To model and forecast asset forward returns using historical and market-driven features.

⚙️ Workflow Overview:

1️⃣ Data preprocessing and feature engineering.

2️⃣ Handling missing values and dropping non-predictive columns like date_id.

3️⃣ Exploratory Data Analysis (EDA) to identify key patterns and trends.

4️⃣ Model training using a Random Forest Regressor for robust prediction.

5️⃣ Evaluation using regression metrics to ensure accuracy and reliability.

6️⃣ Visualization to compare model performance before and after training.

7️⃣ Exporting predictions to submission.csv for further financial insights.

🚀 Outcome: A reliable and interpretable ML model capable of assisting in investment decision-making and risk analysis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error, mean_squared_error, r2_score
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
test = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/test.csv")

print(" Train shape:", train.shape)
print(" Test shape:", test.shape)

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.isnull().sum().sum()

In [ ]:
missing_summary = train.isna().sum().sort_values(ascending=False)
missing_top10 = missing_summary.head(60)
print("\n Top 10 columns containing NaN:")
print(missing_top10)

In [ ]:
nan_counts = train.isna().sum()
for col in train.columns:
    if nan_counts[col] > 1500:
        train = train.drop(columns=[col])

In [ ]:
train.shape

In [ ]:
train.isnull().sum().sum()

In [ ]:
num_cols = train.select_dtypes(include=[np.number]).columns
for col in num_cols:
    if train[col].isna().sum() > 0:
        median_value = train[col].median()
        train[col].fillna(median_value, inplace=True)

In [ ]:
train.isnull().sum().sum()

In [ ]:
train.drop(columns=['date_id'], inplace=True)

In [ ]:
X = train.drop(columns=['forward_returns'])
y = train["forward_returns"]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
    callbacks=[early_stopping(stopping_rounds=100), log_evaluation(100)]
)


In [ ]:
val_pred = model.predict(X_val)

mse = mean_squared_error(y_val, val_pred)
rmse = np.sqrt(mse)

r2 = r2_score(y_val, val_pred)

print(f"Validation MSE: {mse:.6f}")
print(f"Validation RMSE: {rmse:.6f}")
print(f"Validation R² (accuracy): {r2:.4f}")


In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(y_train, bins=50, kde=True, color='skyblue')
plt.title("Distribution of Target (Before Training)")
plt.xlabel("forward_returns")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
sns.scatterplot(x=y_val, y=val_pred, alpha=0.5, color='green')
plt.xlabel("True forward_returns")
plt.ylabel("Predicted forward_returns")
plt.title("True vs Predicted (After Training)")
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--')  
plt.show()


In [ ]:
errors = y_val - val_pred
plt.figure(figsize=(8,5))
sns.histplot(errors, bins=50, kde=True, color='orange')
plt.title("Error Distribution (y_true - y_pred)")
plt.xlabel("Error")
plt.ylabel("Frequency")
plt.axvline(0, color='red', linestyle='--')
plt.show()

In [ ]:
lgb.plot_importance(model, max_num_features=20, figsize=(10,6))
plt.title("Top 20 Important Features")
plt.show()

In [ ]:
target = "forward_returns"

features = [col for col in train.columns if col != target]

for col in features:
    if col not in test.columns:
        test[col] = train[col].mean()

test = test[features]

preds = model.predict(test)

submission = pd.DataFrame({"prediction": preds})
submission.to_csv("/kaggle/working/submission.csv", index=False)
print("✅ The submission.csv file was successfully created")